In [3]:
import sys
sys.path.append('/ossfs/workspace/jd_paper/mydata/')
sys.path.append('/ossfs/workspace/jd_paper/HGT-semi/')
import scipy.io
import urllib.request
import dgl
import math
import numpy as np
import pandas as pd
from model_k import *
import torch
from data_loader import data_loader
from utils.data import load_data
from utils.pytorchtools import EarlyStopping
from utils.tools import evaluate_results_nc
import argparse
import time

import networkx as nx
import random
from collections import defaultdict

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

class Augmenter:
    def __init__(self, args, features_list, is_label):
        print("loading data:")
        dataset = data_loader('/ossfs/workspace/jd_paper/mydata/'+args.dataset)
        self.args = args
        self.features_list = features_list
        self.is_label = is_label
        self.dataset = dataset
        nxg = nx.Graph()
        for i, meta_path in dataset.links["meta"].items():
            nxg.add_edges_from(list(zip(dataset.links["data"][i].tocoo().row, dataset.links["data"][i].tocoo().col)))
        # degree = nx.degree_histogram(nxg)
        # print("nxg", degree)
        # print(nxg.number_of_edges())
        # ('0', '0_1', '1') 9949 9949
        # ('1', '1_0', '0') 9949 9949
        # ('0', '0_2', '2') 3025 3025
        # ('2', '2_0', '0') 3025 3025
        # ('0', '0_0', '0') 5343 5343
        # https://networkx.org/documentation/networkx-1.10/reference/algorithms.link_prediction.html
        nxg.remove_edges_from(list(nx.selfloop_edges(nxg)))
        # 所有1跳节点对（删边）
        aa_edges_1 = list(nx.adamic_adar_index(nxg, nxg.edges()))
        # aa_edges_1 = [item[2] for item in aa_edges_1]
        # print("aa1:", len(aa_edges_1), max(aa_edges_1), min(aa_edges_1), sum(aa_edges_1)/len(aa_edges_1))
        # 所有2跳节点对（增边）
        valid_edge_types = set([(st, dt) for _, (st, dt) in dataset.links["meta"].items()])
        hop2_edges = [(s, d) for s, d in self.hop_two_edges(nxg) if (self.node_type(s), self.node_type(d)) in valid_edge_types]
        aa_edges_2 = list(nx.adamic_adar_index(nxg, hop2_edges))
        # aa_edges_2 = [item[2] for item in aa_edges_2]
        # print("aa2:", len(aa_edges_2), max(aa_edges_2), min(aa_edges_2), sum(aa_edges_2)/len(aa_edges_2))
        # 边采样：1跳节点对正排头部采样删边，2跳节点对倒排头部采样增边
        self.pre_sample_ratio = 1.0 # 采样边占比 (0.8)
        self.sample_ratio = 0.5 # 采样比例
        pre_sample_num = int(dataset.links["total"]*self.pre_sample_ratio)
        sample_num = int(pre_sample_num*self.sample_ratio)
        print("sample_num:", sample_num)
        aa_edges_1 = aa_edges_1+[(item[1],item[0],item[2]) for item in aa_edges_1]
        random.shuffle(aa_edges_1)
        aa_edges_hop1 = sorted(aa_edges_1, key=lambda x:x[-1])[:pre_sample_num]
        aa_edges_2 = aa_edges_2+[(item[1],item[0],item[2]) for item in aa_edges_2]
        random.shuffle(aa_edges_2)
        aa_edges_hop2 = sorted(aa_edges_2, key=lambda x:x[-1], reverse=True)[:pre_sample_num]
        self.sample_num = sample_num
        self.aa_edges_hop1 = aa_edges_hop1
        self.aa_edges_hop2 = aa_edges_hop2
    def hop_two_edges(self, g):
        exist = set(g.edges())
        edges = set()
        for m in g.nodes:
            for a in g.neighbors(m):
                for b in g.neighbors(m):
                    if (a, b) in exist or (b, a) in exist:
                        continue
                    if b > a:
                        edges.add((a, b))
        return edges
    def node_type(self, node):
        for i in range(len(self.dataset.nodes["shift"])-1):
            if node < self.dataset.nodes["shift"][i+1]:
                return i
        else: return i+1
#     def edge_type(self, nodes):
#         for i,items in self.dataset.links['meta'].items():
# #             print('items>>>>',items)
#             if nodes == items:
#                 return i
    def aug_heterograph(self):
        t0 = time.time()
        edges_hop1_rind = np.random.choice(np.arange(len(self.aa_edges_hop1)), size=self.sample_num, replace=False)
        edges_hop1 = [item[:2] for i, item in enumerate(self.aa_edges_hop1) if i in edges_hop1_rind]
        edges_hop2_rind = np.random.choice(np.arange(len(self.aa_edges_hop2)), size=self.sample_num, replace=False)
        edges_hop2 = [item[:2] for i, item in enumerate(self.aa_edges_hop2) if i in edges_hop2_rind] 
        edge_dict_del = defaultdict(list)
        for s, d in edges_hop1:
            st, dt = self.node_type(s), self.node_type(d)
            edge_dict_del[(str(st), str(st)+"_"+str(dt), str(dt))].append((s, d))
        # for k, v in edge_dict_del.items():
        #     print("del", k, len(v))
        edge_dict_add = defaultdict(list)
        for s, d in edges_hop2:
            st, dt = self.node_type(s), self.node_type(d)
            edge_dict_add[(str(st), str(st)+"_"+str(dt), str(dt))].append((s, d))
        # for k, v in edge_dict_add.items():
        #     print("add", k, len(v))
        edge_dict = {}
        edge_dict_weight = {}
        node_degree_dict = defaultdict(int)
        for i, meta_path in self.dataset.links["meta"].items():
            key = (str(meta_path[0]), str(meta_path[0]) + '_' + str(meta_path[1]), str(meta_path[1]))
            ori = list(zip(self.dataset.links["data"][i].tocoo().row, self.dataset.links["data"][i].tocoo().col))
#             edges = set(ori)|set(edge_dict_add[key])
            # edges = set(ori)-set(edge_dict_del[key])
            edges = set(ori)-set(edge_dict_del[key])|set(edge_dict_add[key])
            # print(key, len(set(ori)), len(set(edge_dict_del[key])), len(set(edge_dict_add[key])), len(edges))
            edge_dict[key] = \
            (torch.tensor(np.array([e[0] for e in edges])-self.dataset.nodes["shift"][meta_path[0]]),
             torch.tensor(np.array([e[1] for e in edges])-self.dataset.nodes["shift"][meta_path[1]]))
            
            edge_dict_weight[str(meta_path[0]) + '_' + str(meta_path[1])] = torch.tensor([0]*len(edges))

            for src, dst in edges:
                node_degree_dict[src] += 1
                node_degree_dict[dst] += 1
        degree_dict = defaultdict(int)
        for val in node_degree_dict.values():
            degree_dict[val] += 1
#         print(degree_dict); exit()
        node_count = {}
        for i, count in self.dataset.nodes["count"].items():
            node_count[str(i)] = count
        # print(node_count)
        # print("building dgl.heterograph:")
        G = dgl.heterograph(edge_dict, num_nodes_dict=node_count, device=device)
        G.node_dict = {}
        G.edge_dict = {}
        for ntype in G.ntypes:
            G.node_dict[ntype] = len(G.node_dict)
        for etype in G.etypes:
            G.edge_dict[etype] = len(G.edge_dict)
            G.edges[etype].data["id"] = torch.ones(G.number_of_edges(etype), dtype=torch.long).to(device)*G.edge_dict[etype]
            # ----------------
            G.edges[etype].data['w'] = edge_dict_weight[etype].to(device)
        for ntype in G.ntypes:
            G.nodes[ntype].data["inp"] = self.features_list[int(ntype)]
            if ntype == "0":
                G.nodes[ntype].data["is_label"] = self.is_label.to(device)
            else:
                G.nodes[ntype].data["is_label"] = torch.zeros(len(G.nodes[ntype].data["inp"])).view(-1,1).to(device)
        print("aug heterograph time:", time.time()-t0)
        return G
    def aug_heterograph2(self, model): # 基于三角形全量增边
        t0 = time.time()
#         edges_hop1 = [item[:2] for i, item in enumerate(self.aa_edges_hop1)] 
        edges_hop2 = [item[:2] for i, item in enumerate(self.aa_edges_hop2)] 
#         edge_dict_del = defaultdict(list)
        edge_dict_add = defaultdict(list)
#         for s, d in edges_hop1:
#             st, dt = self.node_type(s), self.node_type(d)
#             edge_dict_del[(str(st), str(st)+"_"+str(dt), str(dt))].append((s, d))
        for s, d in edges_hop2:
            st, dt = self.node_type(s), self.node_type(d)
#             print('etype>>>>>',etype)
            edge_dict_add[(str(st), str(st)+"_"+str(dt), str(dt))].append((s, d))
        edge_dict = {}; edge_dict_info = {}
        edge_dict_weight = {}
        for i, meta_path in self.dataset.links["meta"].items():
            key = (str(meta_path[0]), str(meta_path[0]) + '_' + str(meta_path[1]), str(meta_path[1]))
            ori = list(zip(self.dataset.links["data"][i].tocoo().row, self.dataset.links["data"][i].tocoo().col))
            # edges = set(ori)|set(edge_dict_add[key])
            edges = list(set(ori))+list(set(edge_dict_add[key])-set(ori))
#             edges = set(ori)-set(edge_dict_del[key])|set(edge_dict_add[key])
            edge_dict[key] = \
            (torch.tensor(np.array([e[0] for e in edges])-self.dataset.nodes["shift"][meta_path[0]]),
             torch.tensor(np.array([e[1] for e in edges])-self.dataset.nodes["shift"][meta_path[1]]))
            edge_dict_info[str(meta_path[0]) + '_' + str(meta_path[1])] = (
                key, len(list(set(ori))), len(list(set(edge_dict_add[key])-set(ori))))
        
            edge_dict_weight[str(meta_path[0]) + '_' + str(meta_path[1])] = torch.tensor([0]*len(edges))
            
        node_count = {}
        for i, count in self.dataset.nodes["count"].items():
            node_count[str(i)] = count
        G = dgl.heterograph(edge_dict, num_nodes_dict=node_count, device=device)
        G.node_dict = {}
        G.edge_dict = {}
        for ntype in G.ntypes:
            G.node_dict[ntype] = len(G.node_dict)
        for etype in G.etypes:
            G.edge_dict[etype] = len(G.edge_dict)
            G.edges[etype].data["id"] = torch.ones(G.number_of_edges(etype), dtype=torch.long).to(device)*G.edge_dict[etype]
            # ----------------
            G.edges[etype].data['w'] = edge_dict_weight[etype].to(device)
        for ntype in G.ntypes:
            G.nodes[ntype].data["inp"] = self.features_list[int(ntype)]
            if ntype == "0":
                G.nodes[ntype].data["is_label"] = self.is_label.to(device)
            else:
                G.nodes[ntype].data["is_label"] = torch.zeros(len(G.nodes[ntype].data["inp"])).view(-1,1).to(device)
        # 基于attention权重排序
        # model.eval()
        # print(edge_dict_info)
        edge_dict_aug = {}
        logits, embeddings = model(G, "0", 0)
        for etype in G.etypes:
            att_weights = abs(G.edges[etype].data["a"].detach().cpu().numpy()).mean(axis=1)
            key, ori_len, add_len = edge_dict_info[etype]
            src_nodes, dst_nodes = edge_dict[key]
            # keep_index = np.argsort(att_weights)[::-1][:ori_len].copy()
            keep_index_1 = np.argsort(att_weights[:ori_len])[::-1][:int((1-self.sample_ratio)*ori_len)].copy()
            keep_index_2 = np.argsort(att_weights[ori_len:])[::-1][:int(self.sample_ratio*ori_len)].copy() # add_len
#             keep_index_1 = np.argsort(att_weights[:ori_len])[::-1][:1].copy()
#             keep_index_2 = np.argsort(att_weights[ori_len:])[::-1][:1].copy()
            keep_index = np.concatenate([keep_index_1, keep_index_2])
            edge_dict_aug[key] = (src_nodes[keep_index], dst_nodes[keep_index])
        G = dgl.heterograph(edge_dict_aug, num_nodes_dict=node_count, device=device)
        G.node_dict = {}
        G.edge_dict = {}
        for ntype in G.ntypes:
            G.node_dict[ntype] = len(G.node_dict)
        for etype in G.etypes:
            G.edge_dict[etype] = len(G.edge_dict)
            G.edges[etype].data["id"] = torch.ones(G.number_of_edges(etype), dtype=torch.long).to(device)*G.edge_dict[etype]
            # ----------------
            G.edges[etype].data['w'] = edge_dict_weight[etype].to(device)  
        for ntype in G.ntypes:
            G.nodes[ntype].data["inp"] = self.features_list[int(ntype)]
            if ntype == "0":
                G.nodes[ntype].data["is_label"] = self.is_label.to(device)
            else:
                G.nodes[ntype].data["is_label"] = torch.zeros(len(G.nodes[ntype].data["inp"])).view(-1,1).to(device)
        return G

def sp_to_spt(mat):
    coo = mat.tocoo()
    values = coo.data
    indices = np.vstack((coo.row, coo.col))

    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    shape = coo.shape

    return torch.sparse.FloatTensor(i, v, torch.Size(shape))


def mat2tensor(mat):
    if type(mat) is np.ndarray:
        return torch.from_numpy(mat).type(torch.FloatTensor)
    return sp_to_spt(mat)

def build_heterograph(args):
    
    print('loading data >>>>>>>>>>>>>>>>>>>>>')
    dataset = data_loader('/ossfs/workspace/jd_paper/mydata/'+args.dataset)
    edge_dict = {}
    edge_dict_weight = {}
    for i, meta_path in dataset.links['meta'].items():
#         if (str(meta_path[0]), str(meta_path[0]) + '_' + str(meta_path[1]), str(meta_path[1])) not in edge_dict_tmp0:
#             edge_dict_tmp0[(str(meta_path[0]), str(meta_path[0]) + '_' + str(meta_path[1]), str(meta_path[1]))] = dataset.links['data'][i].tocoo().row - dataset.nodes['shift'][meta_path[0]]
#             edge_dict_tmp1[(str(meta_path[0]), str(meta_path[0]) + '_' + str(meta_path[1]), str(meta_path[1]))] = dataset.links['data'][i].tocoo().col - dataset.nodes['shift'][meta_path[1]]
#         else:
#             edge_dict_tmp0[(str(meta_path[0]), str(meta_path[0]) + '_' + str(meta_path[1]), str(meta_path[1]))] = np.r_[edge_dict_tmp0[(str(meta_path[0]), str(meta_path[0]) + '_' + str(meta_path[1]), str(meta_path[1]))],dataset.links['data'][i].tocoo().row - dataset.nodes['shift'][meta_path[0]]]
#             edge_dict_tmp1[(str(meta_path[0]), str(meta_path[0]) + '_' + str(meta_path[1]), str(meta_path[1]))] = np.r_[edge_dict_tmp1[(str(meta_path[0]), str(meta_path[0]) + '_' + str(meta_path[1]), str(meta_path[1]))],dataset.links['data'][i].tocoo().col - dataset.nodes['shift'][meta_path[1]]]
        
#         edge_dict[(str(meta_path[0]), str(meta_path[0]) + '_' + str(meta_path[1]), str(meta_path[1]))] = (edge_dict_tmp0[(str(meta_path[0]), str(meta_path[0]) + '_' + str(meta_path[1]), str(meta_path[1]))], edge_dict_tmp1[(str(meta_path[0]), str(meta_path[0]) + '_' + str(meta_path[1]), str(meta_path[1]))])
            
#             if (str(meta_path[0]), str(meta_path[0]) + '_' + str(meta_path[1]), str(meta_path[1])) not in edge_dict:
        edge_dict[(str(meta_path[0]), str(meta_path[0]) + '_' + str(meta_path[1]), str(meta_path[1]))] = (torch.tensor(dataset.links['data'][i].tocoo().row - dataset.nodes['shift'][meta_path[0]]), torch.tensor(dataset.links['data'][i].tocoo().col - dataset.nodes['shift'][meta_path[1]]))
        edge_dict_weight[str(meta_path[0]) + '_' + str(meta_path[1])] = torch.tensor(dataset.links['data'][i].tocoo().data)

        
    node_count = {}
    for i, count in dataset.nodes['count'].items():
        print(i, node_count)
        node_count[str(i)] = count

    print('building dgl.heterograph >>>>>>>>>>>>>>>>>>>>>')
    G = dgl.heterograph(edge_dict, num_nodes_dict = node_count, device=device)
    """
    for ntype in G.ntypes:
        G.nodes[ntype].data['inp'] = dataset.nodes['attr'][ntype]
        # print(G.nodes['attr'][ntype].shape)
    """

    G.node_dict = {}
    G.edge_dict = {}
    for ntype in G.ntypes:
        G.node_dict[ntype] = len(G.node_dict)
    for etype in G.etypes:
        G.edge_dict[etype] = len(G.edge_dict)
        G.edges[etype].data['id'] = torch.ones(G.number_of_edges(etype), dtype=torch.long).to(device) * G.edge_dict[etype] 
        print('number_of_edges>>>>>>>>',etype,'**',G.number_of_edges(etype))
        G.edges[etype].data['w'] = edge_dict_weight[etype].to(device)                                                                                                                             
                                                                                                                                     
    return G,dataset

def consis_loss(logps, temp):
    ps = [torch.exp(p) for p in logps]
    sum_p = 0.
    for p in ps:
        sum_p = sum_p + p
    avg_p = sum_p/len(ps)
    #p2 = torch.exp(logp2)
    
    sharp_p = (torch.pow(avg_p, 1./temp) / torch.sum(torch.pow(avg_p, 1./temp), dim=1, keepdim=True)).detach()
    loss = 0.
    for p in ps:
        loss += torch.mean((p-sharp_p).pow(2).sum(1))
    loss = loss/len(ps)
    return args.rate * loss

def run_model_DBLP(args, G, dataset, rep):
    if args.device >= 0:
        device = torch.device("cuda:"+str(args.device))
    else:
        device = torch.device('cpu')
    
    feats_type = args.feats_type
    features_list, adjM, labels, train_val_test_idx, dl = load_data(args.dataset)
    features_list = [mat2tensor(features).to(device) for features in features_list]

    if feats_type == 0:
        in_dims = [features.shape[1] for features in features_list]
    elif feats_type == 1 or feats_type == 5:
        save = 0 if feats_type == 1 else 2
        in_dims = []#[features_list[0].shape[1]] + [10] * (len(features_list) - 1)
        for i in range(0, len(features_list)):
            if i == save:
                in_dims.append(features_list[i].shape[1])
            else:
                in_dims.append(10)
                features_list[i] = torch.zeros((features_list[i].shape[0], 10)).to(device)
    elif feats_type == 2 or feats_type == 4:
        save = feats_type - 2
        in_dims = [features.shape[0] for features in features_list]
        for i in range(0, len(features_list)):
            if i == save:
                in_dims[i] = features_list[i].shape[1]
                continue
            dim = features_list[i].shape[0]
            indices = np.vstack((np.arange(dim), np.arange(dim)))
            indices = torch.LongTensor(indices)
            values = torch.FloatTensor(np.ones(dim))
            features_list[i] = torch.sparse.FloatTensor(indices, values, torch.Size([dim, dim])).to(device)
    elif feats_type == 3:
        in_dims = [features.shape[0] for features in features_list]
        for i in range(len(features_list)):
            dim = features_list[i].shape[0]
            indices = np.vstack((np.arange(dim), np.arange(dim)))
            indices = torch.LongTensor(indices)
            values = torch.FloatTensor(np.ones(dim))
            features_list[i] = torch.sparse.FloatTensor(indices, values, torch.Size([dim, dim])).to(device)

    ## 添加图特征
    for ntype in G.ntypes:
        G.nodes[ntype].data['inp'] = features_list[int(ntype)]#.to(device)
    labels = torch.LongTensor(labels).to(device)
    train_idx = train_val_test_idx['train_idx']
    train_idx = np.sort(train_idx)
    val_idx = train_val_test_idx['val_idx']
    val_idx = np.sort(val_idx)
    test_idx = train_val_test_idx['test_idx']
    test_idx = np.sort(test_idx)
    unlabel_idx = train_val_test_idx['unlabel_idx']
    unlabel_idx = np.sort(unlabel_idx)
    
    ## kk
    is_label = torch.ones(len(G.nodes['0'].data['inp'])).view(-1,1)
    is_label[unlabel_idx]=0
    for ntype in G.ntypes:
        if ntype == '0':
            G.nodes[ntype].data['is_label'] = is_label.to(device)
        else:
            G.nodes[ntype].data['is_label'] = torch.zeros(len(G.nodes[ntype].data['inp'])).view(-1,1).to(device)
    ## kk

    print(len(train_idx), len(test_idx), len(val_idx), len(unlabel_idx))



    model = HGT(G, n_inps=in_dims, n_hid=args.hidden_dim, n_out=labels.max().item()+1, n_layers=args.num_layers, n_heads=args.num_heads, unlabel_idx=unlabel_idx, use_norm = args.use_norm).to(device)
    # if torch.cuda.device_count() > 1:
    #    model = nn.DataParallel(model，device_ids=[0,1])

    # model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), weight_decay=args.weight_decay)#, lr=args.lr, weight_decay=args.weight_decay)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, total_steps=args.schedule_step, max_lr = 1e-3, pct_start=0.05)

    early_stopping = EarlyStopping(patience=args.patience, verbose=True, save_path='/ossfs/workspace/jd_paper/result/tcheckpoint_consis_{}_{}l_{}th.pt'.format(args.dataset, args.num_layers, rep))
    ## kk
    augmenter = Augmenter(args=args, features_list=features_list, is_label=is_label)
    ## kk
    train_step = 0
    for epoch in range(args.epoch):
        with torch.autograd.set_detect_anomaly(True):
            t_start = time.time()
            model.train()

            label_loss = 0
            unlabel_loss_list = []
            for num_cf in args.num_cf_list:  #[0,5,10,15]
                logits, _ = model(G, '0', num_cf)
                label_loss += F.cross_entropy(logits[train_idx], labels[train_idx].to(device))
                unlabel_loss_list.append(torch.log_softmax(logits[unlabel_idx], dim=-1))
                if num_cf==0:
                    pred = logits.argmax(1)
            # ------ 结构增强 ------
            G_aug = augmenter.aug_heterograph()
            logits2, embeddings2 = model(G_aug, "0", 0)
            # ------ ------- ------
            unlabel_loss_list.append(torch.log_softmax(logits2[unlabel_idx], dim=-1))

            unlabel_loss = consis_loss(unlabel_loss_list, temp=args.tem)
            label_loss = label_loss/len(args.num_cf_list)
            print('cross_entropy>>>>>>>> ',label_loss)
            print('consis_loss>>>>>>>> ',unlabel_loss)

            train_loss = label_loss + unlabel_loss
            train_acc = (pred[train_idx] == labels[train_idx]).float().mean()

            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
            train_step += 1
            scheduler.step(train_step)
            t_end = time.time()
        print('Epoch {:05d} | Train_Loss: {:.4f} | Train_acc: {:.4f} |Time: {:.4f}'.format(epoch, train_loss.item(), train_acc.item(), t_end-t_start))
        t_start = time.time()
        model.eval()
        with torch.no_grad():
            logits, val_embeddings = model(G, '0', 0)
            
            val_loss = F.cross_entropy(logits[val_idx], labels[val_idx])
            pred = logits.cpu().numpy().argmax(axis=1)
        val_acc   = (pred[val_idx] == labels[val_idx].cpu().numpy()).mean()
        test_acc  = (pred[test_idx] == labels[test_idx].cpu().numpy()).mean()
        test_pred_ = pred[test_idx]
        
        
        t_end = time.time()
        print('Epoch {:05d} | Val_Loss {:.4f} | Val_acc {:.4f} | Test_acc {:.4f} | Time(s) {:.4f}'.format(
            epoch, val_loss.item(), val_acc.item(), test_acc.item(), t_end - t_start))
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print('Early stopping!')
            return model,test_idx,dataset,dl 
    return model,test_idx,dataset,dl
  
def inference(args, model, test_idx, dataset, G, rep, dl):
    model.load_state_dict(torch.load('/ossfs/workspace/jd_paper/result/tcheckpoint_consis_{}_{}l_{}th.pt'.format(args.dataset, args.num_layers, rep)))
    model.eval()

    with torch.no_grad():
        logits, test_embeddings = model(G, '0', 0)
        test_logits = logits[test_idx]
        
        if args.dataset == 'IMDB1':
            pred = test_logits.cpu().numpy()
        else:
            pred = test_logits.cpu().numpy().argmax(axis=1)
        
        onehot = np.eye(dataset.labels_train['num_classes'], dtype=np.int32)
        pred = onehot[pred]
                
        eval_res = dataset.evaluate(pred)
        print(eval_res['micro-f1'], eval_res['macro-f1'])
        
        svm_macro_f1_list, svm_micro_f1_list, nmi_mean, nmi_std, ari_mean, ari_std = evaluate_results_nc(
                test_embeddings[test_idx].cpu().numpy(), dataset.labels_test['data'][dataset.labels_test['mask']].argmax(axis=1), num_classes=dataset.labels_test['num_classes'])

        return eval_res,svm_macro_f1_list, svm_micro_f1_list, nmi_mean, nmi_std, ari_mean, ari_std

In [ ]:
# if __name__ == '__main__':
ap = argparse.ArgumentParser(description='MRGNN testing for the DBLP dataset')
ap.add_argument('--feats-type', type=int, default=0,
                help='Type of the node features used. ' +
                        '0 - loaded features; ' +
                        '1 - only target node features (zero vec for others); ' +
                        '2 - only target node features (id vec for others); ' +
                        '3 - all id vec. Default is 2;' +
                    '4 - only term features (id vec for others);' + 
                    '5 - only term features (zero vec for others).')
ap.add_argument('--hidden_dim', type=int, default=64, help='Dimension of the node hidden state. Default is 64.')
ap.add_argument('--num_heads', type=int, default=8, help='Number of the attention heads. Default is 8.')
ap.add_argument('--epoch', type=int, default=300, help='Number of epochs.')
ap.add_argument('--patience', type=int, default=30, help='Patience.')
ap.add_argument('--repeat', type=int, default=5, help='Repeat the training and testing for N times. Default is 1.')
ap.add_argument('--num_layers', type=int, default=5) #5
ap.add_argument('--lr', type=float, default=1e-2) #5e-3
ap.add_argument('--dropout', type=float, default=0)
ap.add_argument('--weight-decay', type=float, default=1e-3) #1e-4
ap.add_argument('--slope', type=float, default=0.05)
ap.add_argument('--dataset', type=str, default='JD2')  #  ACM
ap.add_argument('--edge-feats', type=int, default=64)
ap.add_argument('--device', type=int, default=0)
ap.add_argument('--schedule_step', type=int, default=300)
ap.add_argument('--use_norm', type=bool, default=True)
ap.add_argument('--rate', type=float, default=0.2)
ap.add_argument('--num_cf_list', type=list, default=[0,5]) #[0,5,10,15]
ap.add_argument('--tem', type=float, default=0.2)

args = ap.parse_args(args=[])
g, dataset = build_heterograph(args)

# 4种类型的节点统一编号
# labels_test1 = pd.read_csv("../data/"+args.dataset+"/label.dat.test", sep = "\t", header=None,
#                  names=["node_id","node_name","node_type","label"])

micro,macro = [],[]
svm_macro_f1_lists,svm_micro_f1_lists,nmi_mean_list,nmi_std_list,ari_mean_list,ari_std_list = [],[],[],[],[],[]
for r in range(args.repeat):
    print('running the '+str(r)+' times >>>>>>>>>>>>')
#     model,test_idx,dataset,dl = run_model_DBLP(args, g, dataset, r, labels_test1)
    model,test_idx,dataset,dl = run_model_DBLP(args, g, dataset, r)
    eval_res,svm_macro_f1_list, svm_micro_f1_list, nmi_mean, nmi_std, ari_mean, ari_std = inference(args, model, test_idx, dataset, g, r, dl)
    macro.append(eval_res['macro-f1'])
    micro.append(eval_res['micro-f1'])
    svm_macro_f1_lists.append(svm_macro_f1_list)
    svm_micro_f1_lists.append(svm_micro_f1_list)
    nmi_mean_list.append(nmi_mean)
    nmi_std_list.append(nmi_std)
    ari_mean_list.append(ari_mean)
    ari_std_list.append(ari_std)

    
m_macro = np.mean(macro)
std_macro = np.std(macro)
m_micro = np.mean(micro)
std_micro = np.std(micro)
sqrt_n = np.sqrt(len(micro))
print('------------------------------ final result ----------------------------------------')
print('final result macro_f1 >>>>>>>>>> ',m_macro,'+-',1.96*std_macro/sqrt_n)
print('final result micro_f1 >>>>>>>>>> ',m_micro,'+-',1.96*std_micro/sqrt_n)

# print out a summary of the evaluations
svm_macro_f1_lists = np.transpose(np.array(svm_macro_f1_lists), (1, 0, 2))
svm_micro_f1_lists = np.transpose(np.array(svm_micro_f1_lists), (1, 0, 2))
nmi_mean_list = np.array(nmi_mean_list)
nmi_std_list = np.array(nmi_std_list)
ari_mean_list = np.array(ari_mean_list)
ari_std_list = np.array(ari_std_list)
print('----------------------------------------------------------------')
print('SVM tests summary')
print('Macro-F1: ' + ', '.join(['{:.6f}~{:.6f} ({:.1f})'.format(
    macro_f1[:, 0].mean(), macro_f1[:, 1].mean(), train_size) for macro_f1, train_size in
    zip(svm_macro_f1_lists, [0.8, 0.6, 0.4, 0.2])]))
print('Micro-F1: ' + ', '.join(['{:.6f}~{:.6f} ({:.1f})'.format(
    micro_f1[:, 0].mean(), micro_f1[:, 1].mean(), train_size) for micro_f1, train_size in
    zip(svm_micro_f1_lists, [0.8, 0.6, 0.4, 0.2])]))
print('K-means tests summary')
print('NMI: {:.6f}~{:.6f}'.format(nmi_mean_list.mean(), nmi_std_list.mean()))
print('ARI: {:.6f}~{:.6f}'.format(ari_mean_list.mean(), ari_std_list.mean()))


loading data >>>>>>>>>>>>>>>>>>>>>
0 {}
1 {'0': 9999}
building dgl.heterograph >>>>>>>>>>>>>>>>>>>>>
number_of_edges>>>>>>>> 0_0 ** 3
number_of_edges>>>>>>>> 0_1 ** 3036
number_of_edges>>>>>>>> 1_0 ** 2978
number_of_edges>>>>>>>> 1_1 ** 5115
running the 0 times >>>>>>>>>>>>
2400 6999 600 6999
loading data:
sample_num: 90903
aug heterograph time: 10.172754287719727
cross_entropy>>>>>>>>  tensor(0.7328, device='cuda:0', grad_fn=<DivBackward0>)
consis_loss>>>>>>>>  tensor(0.0293, device='cuda:0', grad_fn=<MulBackward0>)


/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch 00000 | Train_Loss: 0.7620 | Train_acc: 0.5192 |Time: 12.9742
Epoch 00000 | Val_Loss 0.6881 | Val_acc 0.5783 | Test_acc 0.5865 | Time(s) 0.1319
Validation loss decreased (inf --> 0.688089).  Saving model ...
aug heterograph time: 10.219975471496582
cross_entropy>>>>>>>>  tensor(0.7308, device='cuda:0', grad_fn=<DivBackward0>)
consis_loss>>>>>>>>  tensor(0.0292, device='cuda:0', grad_fn=<MulBackward0>)
Epoch 00001 | Train_Loss: 0.7599 | Train_acc: 0.5412 |Time: 12.9513
Epoch 00001 | Val_Loss 0.6871 | Val_acc 0.5867 | Test_acc 0.5878 | Time(s) 0.1335
Validation loss decreased (0.688089 --> 0.687061).  Saving model ...
aug heterograph time: 10.03123164176941
cross_entropy>>>>>>>>  tensor(0.7376, device='cuda:0', grad_fn=<DivBackward0>)
consis_loss>>>>>>>>  tensor(0.0296, device='cuda:0', grad_fn=<MulBackward0>)
Epoch 00002 | Train_Loss: 0.7672 | Train_acc: 0.5450 |Time: 13.0398
Epoch 00002 | Val_Loss 0.6856 | Val_acc 0.5917 | Test_acc 0.5907 | Time(s) 0.1400
Validation loss decrease

In [14]:
a = torch.ones(3,4)
b = torch.Tensor([0,1,2])
print(torch.sigmoid(b))
print(b.shape)
print(b.shape[0])
print(b.reshape(b.shape[0],1).shape)

c=b.reshape(b.shape[0],1)
print(a.mul(c))


tensor([0.5000, 0.7311, 0.8808])
torch.Size([3])
3
torch.Size([3, 1])
tensor([[0., 0., 0., 0.],
        [1., 1., 1., 1.],
        [2., 2., 2., 2.]])


In [ ]:

import pandas as pd
from data_loader import gen_file_for_evaluate
data = 'DBLP'

for i in range(5):
    df = pd.read_csv('../result/'+data+'_'+str(i)+'.txt', sep='\t', names=["node_id","node_name","node_type","label"])
    df[['label']].to_csv('../result1/'+data+'_'+str(i)+'.txt', sep='\t',index=False, header = False)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
data = pd.read_csv('../SemiData/kmeans_res_ACM.csv')

emb = data.drop(['y_pred'],axis=1)
y_pred = data['y_pred'].values

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
# pca = PCA(n_components=2)
# pca.fit(emb)
# X_new = pca.transform(emb)

tsne = TSNE(n_components=2)
tsne.fit(emb)
X_new = tsne.transform(emb)

mark=['or','+b','sg','dk','py']
j=0
for i in y_pred:
    plt.plot(X_new[j:j+1,0],X_new[j:j+1,1],mark[i],markersize=2)
    j += 1
plt.show()